In [1]:
from datetime import datetime
from metloom.pointdata import SnotelPointData, CDECPointData
from metloom.variables import SnotelVariables, CdecStationVariables
import geopandas as gpd
from pathlib import Path
from utils.read_files import read_from_control, make_default_path

In [2]:
# Store the name of the 'active' file in a variable
controlFile = 'control_EastRiver.txt'
# basin name and outlet gauge id
root_path = Path(read_from_control(controlFile, "root_path"))
name = read_from_control(controlFile, "domain_name")

fp = root_path / f"domain_{name}"
obj = gpd.read_file(fp / "shapefiles" / "catchment" / f"{name}.shp")

In [3]:
vrs = [
    SnotelVariables.SWE,
    SnotelVariables.SNOWDEPTH
]
points = SnotelPointData.points_from_geometry(obj, vrs)
gdf = points.to_dataframe()

In [37]:
# download the data from each site
for i, id in enumerate(gdf.id):
    snotel_point = SnotelPointData(id, "Station")
    print(f"Downloading data for {id}...")
    df = snotel_point.get_daily_data(
        datetime(2021, 10, 1), datetime(2023, 9, 30),
        [snotel_point.ALLOWED_VARIABLES.SWE,
         snotel_point.ALLOWED_VARIABLES.PRECIPITATION,
         snotel_point.ALLOWED_VARIABLES.PRECIPITATIONACCUM,
         snotel_point.ALLOWED_VARIABLES.SOILMOISTURE2IN,
         snotel_point.ALLOWED_VARIABLES.SOILMOISTURE8IN,
         snotel_point.ALLOWED_VARIABLES.SOILMOISTURE20IN,
         snotel_point.ALLOWED_VARIABLES.TEMP,
         snotel_point.ALLOWED_VARIABLES.RH,
         ]
    )
    if df is None:
        continue
    else:
        # save the data to a csv file
        filename = 'OBS_' + gdf.loc[i]['name'].replace(" ", "_").upper()
        df.to_csv(fp / "forcing" / "01_raw_data" / f"{filename}.csv")

No Relative Humidity found for Station


In [ ]:
vrs = [
    CdecStationVariables.SWE,
    CdecStationVariables.SNOWDEPTH
]
points = CDECPointData.points_from_geometry(obj, vrs)
gdf = points.to_dataframe()

In [33]:
# download the data from each site
for i, id in enumerate(gdf.id):
    snotel_point = CDECPointData(id, "Station")
    print(f"Downloading data for {id}...")
    df = snotel_point.get_daily_data(
        datetime(2021, 10, 1), datetime(2023, 9, 30),
        [snotel_point.ALLOWED_VARIABLES.SWE,
         snotel_point.ALLOWED_VARIABLES.PRECIPITATION,
         snotel_point.ALLOWED_VARIABLES.PRECIPITATIONACCUM,
        #  snotel_point.ALLOWED_VARIABLES.SOILMOISTURE2IN,
        #  snotel_point.ALLOWED_VARIABLES.SOILMOISTURE4IN,
        #  snotel_point.ALLOWED_VARIABLES.SOILMOISTURE8IN,
        #  snotel_point.ALLOWED_VARIABLES.SOILMOISTURE20IN,
         snotel_point.ALLOWED_VARIABLES.TEMP,
         snotel_point.ALLOWED_VARIABLES.RH,
         ]
    )
    if df is None:
        continue
    else:
        # save the data to a csv file
        filename = 'OBS_' + gdf.loc[i]['name'].replace(" ", "_").upper()
        df.to_csv(fp / f"{filename}.csv")